# Linear regression

Created 48x12x2 features, 1 feature for each hour, each state of workingday (0 or 1), each month and each year.
Clean the data by removing data in the which are over the 95% quantile or under the 5%
fit the linear regression on the clean data

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0,0,1,1


In [2]:
predictors = ['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'hour']
targets = ["count"]

train.corr() #linear correlation between features

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
season,1.000000,0.029368,-0.008126,0.008879,0.258689,0.264744,0.190610,-0.147121,0.096758,0.164011,0.163439
holiday,0.029368,1.000000,-0.250491,-0.007074,0.000295,-0.005215,0.001929,0.008409,0.043799,-0.020956,-0.005393
workingday,-0.008126,-0.250491,1.000000,0.033772,0.029966,0.024660,-0.010880,0.013373,-0.319111,0.119460,0.011594
weather,0.008879,-0.007074,0.033772,1.000000,-0.055035,-0.055376,0.406244,0.007261,-0.135918,-0.109340,-0.128655
temp,0.258689,0.000295,0.029966,-0.055035,1.000000,0.984948,-0.064949,-0.017852,0.467097,0.318571,0.394454
atemp,0.264744,-0.005215,0.024660,-0.055376,0.984948,1.000000,-0.043536,-0.057473,0.462067,0.314635,0.389784
humidity,0.190610,0.001929,-0.010880,0.406244,-0.064949,-0.043536,1.000000,-0.318607,-0.348187,-0.265458,-0.317371
windspeed,-0.147121,0.008409,0.013373,0.007261,-0.017852,-0.057473,-0.318607,1.000000,0.092276,0.091052,0.101369
casual,0.096758,0.043799,-0.319111,-0.135918,0.467097,0.462067,-0.348187,0.092276,1.000000,0.497250,0.690414
registered,0.164011,-0.020956,0.119460,-0.109340,0.318571,0.314635,-0.265458,0.091052,0.497250,1.000000,0.970948


## Create new features that can be used with linear regression

In [3]:
def splitDatetime(data, columndate) :
    datatime= pd.to_datetime(data[columndate])
    data['year'] = datatime.dt.year
    data['month'] = datatime.dt.month
    data['day'] = datatime.dt.day
    data['dayoftheweek'] = datatime.dt.dayofweek
    data['hour'] = datatime.dt.hour
    return data

def harmonize(data):
    data.loc[:,['temp', 'atemp', 'windspeed']]= data.loc[:,['temp', 'atemp', 'windspeed']].round(2)
    return

def normalizedata(data):
    for col in predictors:
        try:
            if data.loc[:, col].mean()>3:
                print(data.loc[:, col].max(), col)
                data.loc[:, col]=(data.loc[:, col]-data.loc[:, col].mean())/data.loc[:, col].std()
        except:
            pass


def RMSLE(predictions, targets):
    return np.sqrt(1/len(predictions)*sum(np.square(np.log(predictions+1)-(np.log(targets+1)))))
    

splitDatetime(train, "datetime")

def createtimefeatures(data):
    for y in range(2011, 2013, 1):
        for m in range(1, 13, 1):
            for x in range(0, 24, 1):
            #     print(quantileup, quantilelow)
                name1 = 'hour '+'{:02d}'.format(x)+'{:02d}'.format(m)+'{:02d}'.format(y)
                name2 = 'hour w'+'{:02d}'.format(x)+'{:02d}'.format(m)+'{:02d}'.format(y)
                data[name1] = 0
                data[name2] = 0
                data.loc[(data['month']==m) &(data['year']==y) & (data['hour']==x) & (data['workingday']==0), name1] = 1 
                data.loc[(data['month']==m) &(data['year']==y) & (data['hour']==x) & (data['workingday']==1), name2] = 1

#Reduce the number of sample by removing outliers in order to increase quality of results and decrease risk of overfitting.                
def reduce(data, column):
    frames = []
    for x in range(0, 24, 1):
        quantileup = data.loc[(data['hour']==x),  column].quantile(0.05)
        quantilelow = data.loc[(data['hour']==x),  column].quantile(0.95)
        frames.append(data.loc[(data['hour']==x) & (data[ column]>quantileup) & (data[ column]<quantilelow), :])
    return pd.concat(frames)
    

createtimefeatures(train)    
result = reduce(train, "count")

splitDatetime(test, "datetime")
createtimefeatures(test)
print("end")

end


# Linear regression

In [5]:
predictors = ['hour 00012011', 'hour w00012011', 'hour 01012011', 'hour w01012011', 'hour 02012011', 'hour w02012011', 'hour 03012011', 'hour w03012011', 'hour 04012011', 'hour w04012011', 'hour 05012011', 'hour w05012011', 'hour 06012011', 'hour w06012011', 'hour 07012011', 'hour w07012011', 'hour 08012011', 'hour w08012011', 'hour 09012011', 'hour w09012011', 'hour 10012011', 'hour w10012011', 'hour 11012011', 'hour w11012011', 'hour 12012011', 'hour w12012011', 'hour 13012011', 'hour w13012011', 'hour 14012011', 'hour w14012011', 'hour 15012011', 'hour w15012011', 'hour 16012011', 'hour w16012011', 'hour 17012011', 'hour w17012011', 'hour 18012011', 'hour w18012011', 'hour 19012011', 'hour w19012011', 'hour 20012011', 'hour w20012011', 'hour 21012011', 'hour w21012011', 'hour 22012011', 'hour w22012011', 'hour 23012011', 'hour w23012011', 'hour 00022011', 'hour w00022011', 'hour 01022011', 'hour w01022011', 'hour 02022011', 'hour w02022011', 'hour 03022011', 'hour w03022011', 'hour 04022011', 'hour w04022011', 'hour 05022011', 'hour w05022011', 'hour 06022011', 'hour w06022011', 'hour 07022011', 'hour w07022011', 'hour 08022011', 'hour w08022011', 'hour 09022011', 'hour w09022011', 'hour 10022011', 'hour w10022011', 'hour 11022011', 'hour w11022011', 'hour 12022011', 'hour w12022011', 'hour 13022011', 'hour w13022011', 'hour 14022011', 'hour w14022011', 'hour 15022011', 'hour w15022011', 'hour 16022011', 'hour w16022011', 'hour 17022011', 'hour w17022011', 'hour 18022011', 'hour w18022011', 'hour 19022011', 'hour w19022011', 'hour 20022011', 'hour w20022011', 'hour 21022011', 'hour w21022011', 'hour 22022011', 'hour w22022011', 'hour 23022011', 'hour w23022011', 'hour 00032011', 'hour w00032011', 'hour 01032011', 'hour w01032011', 'hour 02032011', 'hour w02032011', 'hour 03032011', 'hour w03032011', 'hour 04032011', 'hour w04032011', 'hour 05032011', 'hour w05032011', 'hour 06032011', 'hour w06032011', 'hour 07032011', 'hour w07032011', 'hour 08032011', 'hour w08032011', 'hour 09032011', 'hour w09032011', 'hour 10032011', 'hour w10032011', 'hour 11032011', 'hour w11032011', 'hour 12032011', 'hour w12032011', 'hour 13032011', 'hour w13032011', 'hour 14032011', 'hour w14032011', 'hour 15032011', 'hour w15032011', 'hour 16032011', 'hour w16032011', 'hour 17032011', 'hour w17032011', 'hour 18032011', 'hour w18032011', 'hour 19032011', 'hour w19032011', 'hour 20032011', 'hour w20032011', 'hour 21032011', 'hour w21032011', 'hour 22032011', 'hour w22032011', 'hour 23032011', 'hour w23032011', 'hour 00042011', 'hour w00042011', 'hour 01042011', 'hour w01042011', 'hour 02042011', 'hour w02042011', 'hour 03042011', 'hour w03042011', 'hour 04042011', 'hour w04042011', 'hour 05042011', 'hour w05042011', 'hour 06042011', 'hour w06042011', 'hour 07042011', 'hour w07042011', 'hour 08042011', 'hour w08042011', 'hour 09042011', 'hour w09042011', 'hour 10042011', 'hour w10042011', 'hour 11042011', 'hour w11042011', 'hour 12042011', 'hour w12042011', 'hour 13042011', 'hour w13042011', 'hour 14042011', 'hour w14042011', 'hour 15042011', 'hour w15042011', 'hour 16042011', 'hour w16042011', 'hour 17042011', 'hour w17042011', 'hour 18042011', 'hour w18042011', 'hour 19042011', 'hour w19042011', 'hour 20042011', 'hour w20042011', 'hour 21042011', 'hour w21042011', 'hour 22042011', 'hour w22042011', 'hour 23042011', 'hour w23042011', 'hour 00052011', 'hour w00052011', 'hour 01052011', 'hour w01052011', 'hour 02052011', 'hour w02052011', 'hour 03052011', 'hour w03052011', 'hour 04052011', 'hour w04052011', 'hour 05052011', 'hour w05052011', 'hour 06052011', 'hour w06052011', 'hour 07052011', 'hour w07052011', 'hour 08052011', 'hour w08052011', 'hour 09052011', 'hour w09052011', 'hour 10052011', 'hour w10052011', 'hour 11052011', 'hour w11052011', 'hour 12052011', 'hour w12052011', 'hour 13052011', 'hour w13052011', 'hour 14052011', 'hour w14052011', 'hour 15052011', 'hour w15052011', 'hour 16052011', 'hour w16052011', 'hour 17052011', 'hour w17052011', 'hour 18052011', 'hour w18052011', 'hour 19052011', 'hour w19052011', 'hour 20052011', 'hour w20052011', 'hour 21052011', 'hour w21052011', 'hour 22052011', 'hour w22052011', 'hour 23052011', 'hour w23052011', 'hour 00062011', 'hour w00062011', 'hour 01062011', 'hour w01062011', 'hour 02062011', 'hour w02062011', 'hour 03062011', 'hour w03062011', 'hour 04062011', 'hour w04062011', 'hour 05062011', 'hour w05062011', 'hour 06062011', 'hour w06062011', 'hour 07062011', 'hour w07062011', 'hour 08062011', 'hour w08062011', 'hour 09062011', 'hour w09062011', 'hour 10062011', 'hour w10062011', 'hour 11062011', 'hour w11062011', 'hour 12062011', 'hour w12062011', 'hour 13062011', 'hour w13062011', 'hour 14062011', 'hour w14062011', 'hour 15062011', 'hour w15062011', 'hour 16062011', 'hour w16062011', 'hour 17062011', 'hour w17062011', 'hour 18062011', 'hour w18062011', 'hour 19062011', 'hour w19062011', 'hour 20062011', 'hour w20062011', 'hour 21062011', 'hour w21062011', 'hour 22062011', 'hour w22062011', 'hour 23062011', 'hour w23062011', 'hour 00072011', 'hour w00072011', 'hour 01072011', 'hour w01072011', 'hour 02072011', 'hour w02072011', 'hour 03072011', 'hour w03072011', 'hour 04072011', 'hour w04072011', 'hour 05072011', 'hour w05072011', 'hour 06072011', 'hour w06072011', 'hour 07072011', 'hour w07072011', 'hour 08072011', 'hour w08072011', 'hour 09072011', 'hour w09072011', 'hour 10072011', 'hour w10072011', 'hour 11072011', 'hour w11072011', 'hour 12072011', 'hour w12072011', 'hour 13072011', 'hour w13072011', 'hour 14072011', 'hour w14072011', 'hour 15072011', 'hour w15072011', 'hour 16072011', 'hour w16072011', 'hour 17072011', 'hour w17072011', 'hour 18072011', 'hour w18072011', 'hour 19072011', 'hour w19072011', 'hour 20072011', 'hour w20072011', 'hour 21072011', 'hour w21072011', 'hour 22072011', 'hour w22072011', 'hour 23072011', 'hour w23072011', 'hour 00082011', 'hour w00082011', 'hour 01082011', 'hour w01082011', 'hour 02082011', 'hour w02082011', 'hour 03082011', 'hour w03082011', 'hour 04082011', 'hour w04082011', 'hour 05082011', 'hour w05082011', 'hour 06082011', 'hour w06082011', 'hour 07082011', 'hour w07082011', 'hour 08082011', 'hour w08082011', 'hour 09082011', 'hour w09082011', 'hour 10082011', 'hour w10082011', 'hour 11082011', 'hour w11082011', 'hour 12082011', 'hour w12082011', 'hour 13082011', 'hour w13082011', 'hour 14082011', 'hour w14082011', 'hour 15082011', 'hour w15082011', 'hour 16082011', 'hour w16082011', 'hour 17082011', 'hour w17082011', 'hour 18082011', 'hour w18082011', 'hour 19082011', 'hour w19082011', 'hour 20082011', 'hour w20082011', 'hour 21082011', 'hour w21082011', 'hour 22082011', 'hour w22082011', 'hour 23082011', 'hour w23082011', 'hour 00092011', 'hour w00092011', 'hour 01092011', 'hour w01092011', 'hour 02092011', 'hour w02092011', 'hour 03092011', 'hour w03092011', 'hour 04092011', 'hour w04092011', 'hour 05092011', 'hour w05092011', 'hour 06092011', 'hour w06092011', 'hour 07092011', 'hour w07092011', 'hour 08092011', 'hour w08092011', 'hour 09092011', 'hour w09092011', 'hour 10092011', 'hour w10092011', 'hour 11092011', 'hour w11092011', 'hour 12092011', 'hour w12092011', 'hour 13092011', 'hour w13092011', 'hour 14092011', 'hour w14092011', 'hour 15092011', 'hour w15092011', 'hour 16092011', 'hour w16092011', 'hour 17092011', 'hour w17092011', 'hour 18092011', 'hour w18092011', 'hour 19092011', 'hour w19092011', 'hour 20092011', 'hour w20092011', 'hour 21092011', 'hour w21092011', 'hour 22092011', 'hour w22092011', 'hour 23092011', 'hour w23092011', 'hour 00102011', 'hour w00102011', 'hour 01102011', 'hour w01102011', 'hour 02102011', 'hour w02102011', 'hour 03102011', 'hour w03102011', 'hour 04102011', 'hour w04102011', 'hour 05102011', 'hour w05102011', 'hour 06102011', 'hour w06102011', 'hour 07102011', 'hour w07102011', 'hour 08102011', 'hour w08102011', 'hour 09102011', 'hour w09102011', 'hour 10102011', 'hour w10102011', 'hour 11102011', 'hour w11102011', 'hour 12102011', 'hour w12102011', 'hour 13102011', 'hour w13102011', 'hour 14102011', 'hour w14102011', 'hour 15102011', 'hour w15102011', 'hour 16102011', 'hour w16102011', 'hour 17102011', 'hour w17102011', 'hour 18102011', 'hour w18102011', 'hour 19102011', 'hour w19102011', 'hour 20102011', 'hour w20102011', 'hour 21102011', 'hour w21102011', 'hour 22102011', 'hour w22102011', 'hour 23102011', 'hour w23102011', 'hour 00112011', 'hour w00112011', 'hour 01112011', 'hour w01112011', 'hour 02112011', 'hour w02112011', 'hour 03112011', 'hour w03112011', 'hour 04112011', 'hour w04112011', 'hour 05112011', 'hour w05112011', 'hour 06112011', 'hour w06112011', 'hour 07112011', 'hour w07112011', 'hour 08112011', 'hour w08112011', 'hour 09112011', 'hour w09112011', 'hour 10112011', 'hour w10112011', 'hour 11112011', 'hour w11112011', 'hour 12112011', 'hour w12112011', 'hour 13112011', 'hour w13112011', 'hour 14112011', 'hour w14112011', 'hour 15112011', 'hour w15112011', 'hour 16112011', 'hour w16112011', 'hour 17112011', 'hour w17112011', 'hour 18112011', 'hour w18112011', 'hour 19112011', 'hour w19112011', 'hour 20112011', 'hour w20112011', 'hour 21112011', 'hour w21112011', 'hour 22112011', 'hour w22112011', 'hour 23112011', 'hour w23112011', 'hour 00122011', 'hour w00122011', 'hour 01122011', 'hour w01122011', 'hour 02122011', 'hour w02122011', 'hour 03122011', 'hour w03122011', 'hour 04122011', 'hour w04122011', 'hour 05122011', 'hour w05122011', 'hour 06122011', 'hour w06122011', 'hour 07122011', 'hour w07122011', 'hour 08122011', 'hour w08122011', 'hour 09122011', 'hour w09122011', 'hour 10122011', 'hour w10122011', 'hour 11122011', 'hour w11122011', 'hour 12122011', 'hour w12122011', 'hour 13122011', 'hour w13122011', 'hour 14122011', 'hour w14122011', 'hour 15122011', 'hour w15122011', 'hour 16122011', 'hour w16122011', 'hour 17122011', 'hour w17122011', 'hour 18122011', 'hour w18122011', 'hour 19122011', 'hour w19122011', 'hour 20122011', 'hour w20122011', 'hour 21122011', 'hour w21122011', 'hour 22122011', 'hour w22122011', 'hour 23122011', 'hour w23122011', 'hour 00012012', 'hour w00012012', 'hour 01012012', 'hour w01012012', 'hour 02012012', 'hour w02012012', 'hour 03012012', 'hour w03012012', 'hour 04012012', 'hour w04012012', 'hour 05012012', 'hour w05012012', 'hour 06012012', 'hour w06012012', 'hour 07012012', 'hour w07012012', 'hour 08012012', 'hour w08012012', 'hour 09012012', 'hour w09012012', 'hour 10012012', 'hour w10012012', 'hour 11012012', 'hour w11012012', 'hour 12012012', 'hour w12012012', 'hour 13012012', 'hour w13012012', 'hour 14012012', 'hour w14012012', 'hour 15012012', 'hour w15012012', 'hour 16012012', 'hour w16012012', 'hour 17012012', 'hour w17012012', 'hour 18012012', 'hour w18012012', 'hour 19012012', 'hour w19012012', 'hour 20012012', 'hour w20012012', 'hour 21012012', 'hour w21012012', 'hour 22012012', 'hour w22012012', 'hour 23012012', 'hour w23012012', 'hour 00022012', 'hour w00022012', 'hour 01022012', 'hour w01022012', 'hour 02022012', 'hour w02022012', 'hour 03022012', 'hour w03022012', 'hour 04022012', 'hour w04022012', 'hour 05022012', 'hour w05022012', 'hour 06022012', 'hour w06022012', 'hour 07022012', 'hour w07022012', 'hour 08022012', 'hour w08022012', 'hour 09022012', 'hour w09022012', 'hour 10022012', 'hour w10022012', 'hour 11022012', 'hour w11022012', 'hour 12022012', 'hour w12022012', 'hour 13022012', 'hour w13022012', 'hour 14022012', 'hour w14022012', 'hour 15022012', 'hour w15022012', 'hour 16022012', 'hour w16022012', 'hour 17022012', 'hour w17022012', 'hour 18022012', 'hour w18022012', 'hour 19022012', 'hour w19022012', 'hour 20022012', 'hour w20022012', 'hour 21022012', 'hour w21022012', 'hour 22022012', 'hour w22022012', 'hour 23022012', 'hour w23022012', 'hour 00032012', 'hour w00032012', 'hour 01032012', 'hour w01032012', 'hour 02032012', 'hour w02032012', 'hour 03032012', 'hour w03032012', 'hour 04032012', 'hour w04032012', 'hour 05032012', 'hour w05032012', 'hour 06032012', 'hour w06032012', 'hour 07032012', 'hour w07032012', 'hour 08032012', 'hour w08032012', 'hour 09032012', 'hour w09032012', 'hour 10032012', 'hour w10032012', 'hour 11032012', 'hour w11032012', 'hour 12032012', 'hour w12032012', 'hour 13032012', 'hour w13032012', 'hour 14032012', 'hour w14032012', 'hour 15032012', 'hour w15032012', 'hour 16032012', 'hour w16032012', 'hour 17032012', 'hour w17032012', 'hour 18032012', 'hour w18032012', 'hour 19032012', 'hour w19032012', 'hour 20032012', 'hour w20032012', 'hour 21032012', 'hour w21032012', 'hour 22032012', 'hour w22032012', 'hour 23032012', 'hour w23032012', 'hour 00042012', 'hour w00042012', 'hour 01042012', 'hour w01042012', 'hour 02042012', 'hour w02042012', 'hour 03042012', 'hour w03042012', 'hour 04042012', 'hour w04042012', 'hour 05042012', 'hour w05042012', 'hour 06042012', 'hour w06042012', 'hour 07042012', 'hour w07042012', 'hour 08042012', 'hour w08042012', 'hour 09042012', 'hour w09042012', 'hour 10042012', 'hour w10042012', 'hour 11042012', 'hour w11042012', 'hour 12042012', 'hour w12042012', 'hour 13042012', 'hour w13042012', 'hour 14042012', 'hour w14042012', 'hour 15042012', 'hour w15042012', 'hour 16042012', 'hour w16042012', 'hour 17042012', 'hour w17042012', 'hour 18042012', 'hour w18042012', 'hour 19042012', 'hour w19042012', 'hour 20042012', 'hour w20042012', 'hour 21042012', 'hour w21042012', 'hour 22042012', 'hour w22042012', 'hour 23042012', 'hour w23042012', 'hour 00052012', 'hour w00052012', 'hour 01052012', 'hour w01052012', 'hour 02052012', 'hour w02052012', 'hour 03052012', 'hour w03052012', 'hour 04052012', 'hour w04052012', 'hour 05052012', 'hour w05052012', 'hour 06052012', 'hour w06052012', 'hour 07052012', 'hour w07052012', 'hour 08052012', 'hour w08052012', 'hour 09052012', 'hour w09052012', 'hour 10052012', 'hour w10052012', 'hour 11052012', 'hour w11052012', 'hour 12052012', 'hour w12052012', 'hour 13052012', 'hour w13052012', 'hour 14052012', 'hour w14052012', 'hour 15052012', 'hour w15052012', 'hour 16052012', 'hour w16052012', 'hour 17052012', 'hour w17052012', 'hour 18052012', 'hour w18052012', 'hour 19052012', 'hour w19052012', 'hour 20052012', 'hour w20052012', 'hour 21052012', 'hour w21052012', 'hour 22052012', 'hour w22052012', 'hour 23052012', 'hour w23052012', 'hour 00062012', 'hour w00062012', 'hour 01062012', 'hour w01062012', 'hour 02062012', 'hour w02062012', 'hour 03062012', 'hour w03062012', 'hour 04062012', 'hour w04062012', 'hour 05062012', 'hour w05062012', 'hour 06062012', 'hour w06062012', 'hour 07062012', 'hour w07062012', 'hour 08062012', 'hour w08062012', 'hour 09062012', 'hour w09062012', 'hour 10062012', 'hour w10062012', 'hour 11062012', 'hour w11062012', 'hour 12062012', 'hour w12062012', 'hour 13062012', 'hour w13062012', 'hour 14062012', 'hour w14062012', 'hour 15062012', 'hour w15062012', 'hour 16062012', 'hour w16062012', 'hour 17062012', 'hour w17062012', 'hour 18062012', 'hour w18062012', 'hour 19062012', 'hour w19062012', 'hour 20062012', 'hour w20062012', 'hour 21062012', 'hour w21062012', 'hour 22062012', 'hour w22062012', 'hour 23062012', 'hour w23062012', 'hour 00072012', 'hour w00072012', 'hour 01072012', 'hour w01072012', 'hour 02072012', 'hour w02072012', 'hour 03072012', 'hour w03072012', 'hour 04072012', 'hour w04072012', 'hour 05072012', 'hour w05072012', 'hour 06072012', 'hour w06072012', 'hour 07072012', 'hour w07072012', 'hour 08072012', 'hour w08072012', 'hour 09072012', 'hour w09072012', 'hour 10072012', 'hour w10072012', 'hour 11072012', 'hour w11072012', 'hour 12072012', 'hour w12072012', 'hour 13072012', 'hour w13072012', 'hour 14072012', 'hour w14072012', 'hour 15072012', 'hour w15072012', 'hour 16072012', 'hour w16072012', 'hour 17072012', 'hour w17072012', 'hour 18072012', 'hour w18072012', 'hour 19072012', 'hour w19072012', 'hour 20072012', 'hour w20072012', 'hour 21072012', 'hour w21072012', 'hour 22072012', 'hour w22072012', 'hour 23072012', 'hour w23072012', 'hour 00082012', 'hour w00082012', 'hour 01082012', 'hour w01082012', 'hour 02082012', 'hour w02082012', 'hour 03082012', 'hour w03082012', 'hour 04082012', 'hour w04082012', 'hour 05082012', 'hour w05082012', 'hour 06082012', 'hour w06082012', 'hour 07082012', 'hour w07082012', 'hour 08082012', 'hour w08082012', 'hour 09082012', 'hour w09082012', 'hour 10082012', 'hour w10082012', 'hour 11082012', 'hour w11082012', 'hour 12082012', 'hour w12082012', 'hour 13082012', 'hour w13082012', 'hour 14082012', 'hour w14082012', 'hour 15082012', 'hour w15082012', 'hour 16082012', 'hour w16082012', 'hour 17082012', 'hour w17082012', 'hour 18082012', 'hour w18082012', 'hour 19082012', 'hour w19082012', 'hour 20082012', 'hour w20082012', 'hour 21082012', 'hour w21082012', 'hour 22082012', 'hour w22082012', 'hour 23082012', 'hour w23082012', 'hour 00092012', 'hour w00092012', 'hour 01092012', 'hour w01092012', 'hour 02092012', 'hour w02092012', 'hour 03092012', 'hour w03092012', 'hour 04092012', 'hour w04092012', 'hour 05092012', 'hour w05092012', 'hour 06092012', 'hour w06092012', 'hour 07092012', 'hour w07092012', 'hour 08092012', 'hour w08092012', 'hour 09092012', 'hour w09092012', 'hour 10092012', 'hour w10092012', 'hour 11092012', 'hour w11092012', 'hour 12092012', 'hour w12092012', 'hour 13092012', 'hour w13092012', 'hour 14092012', 'hour w14092012', 'hour 15092012', 'hour w15092012', 'hour 16092012', 'hour w16092012', 'hour 17092012', 'hour w17092012', 'hour 18092012', 'hour w18092012', 'hour 19092012', 'hour w19092012', 'hour 20092012', 'hour w20092012', 'hour 21092012', 'hour w21092012', 'hour 22092012', 'hour w22092012', 'hour 23092012', 'hour w23092012', 'hour 00102012', 'hour w00102012', 'hour 01102012', 'hour w01102012', 'hour 02102012', 'hour w02102012', 'hour 03102012', 'hour w03102012', 'hour 04102012', 'hour w04102012', 'hour 05102012', 'hour w05102012', 'hour 06102012', 'hour w06102012', 'hour 07102012', 'hour w07102012', 'hour 08102012', 'hour w08102012', 'hour 09102012', 'hour w09102012', 'hour 10102012', 'hour w10102012', 'hour 11102012', 'hour w11102012', 'hour 12102012', 'hour w12102012', 'hour 13102012', 'hour w13102012', 'hour 14102012', 'hour w14102012', 'hour 15102012', 'hour w15102012', 'hour 16102012', 'hour w16102012', 'hour 17102012', 'hour w17102012', 'hour 18102012', 'hour w18102012', 'hour 19102012', 'hour w19102012', 'hour 20102012', 'hour w20102012', 'hour 21102012', 'hour w21102012', 'hour 22102012', 'hour w22102012', 'hour 23102012', 'hour w23102012', 'hour 00112012', 'hour w00112012', 'hour 01112012', 'hour w01112012', 'hour 02112012', 'hour w02112012', 'hour 03112012', 'hour w03112012', 'hour 04112012', 'hour w04112012', 'hour 05112012', 'hour w05112012', 'hour 06112012', 'hour w06112012', 'hour 07112012', 'hour w07112012', 'hour 08112012', 'hour w08112012', 'hour 09112012', 'hour w09112012', 'hour 10112012', 'hour w10112012', 'hour 11112012', 'hour w11112012', 'hour 12112012', 'hour w12112012', 'hour 13112012', 'hour w13112012', 'hour 14112012', 'hour w14112012', 'hour 15112012', 'hour w15112012', 'hour 16112012', 'hour w16112012', 'hour 17112012', 'hour w17112012', 'hour 18112012', 'hour w18112012', 'hour 19112012', 'hour w19112012', 'hour 20112012', 'hour w20112012', 'hour 21112012', 'hour w21112012', 'hour 22112012', 'hour w22112012', 'hour 23112012', 'hour w23112012', 'hour 00122012', 'hour w00122012', 'hour 01122012', 'hour w01122012', 'hour 02122012', 'hour w02122012', 'hour 03122012', 'hour w03122012', 'hour 04122012', 'hour w04122012', 'hour 05122012', 'hour w05122012', 'hour 06122012', 'hour w06122012', 'hour 07122012', 'hour w07122012', 'hour 08122012', 'hour w08122012', 'hour 09122012', 'hour w09122012', 'hour 10122012', 'hour w10122012', 'hour 11122012', 'hour w11122012', 'hour 12122012', 'hour w12122012', 'hour 13122012', 'hour w13122012', 'hour 14122012', 'hour w14122012', 'hour 15122012', 'hour w15122012', 'hour 16122012', 'hour w16122012', 'hour 17122012', 'hour w17122012', 'hour 18122012', 'hour w18122012', 'hour 19122012', 'hour w19122012', 'hour 20122012', 'hour w20122012', 'hour 21122012', 'hour w21122012', 'hour 22122012', 'hour w22122012', 'hour 23122012', 'hour w23122012']
targets = ["count"]

## Train algorithm

In [6]:
from sklearn.cross_validation import KFold
from sklearn.linear_model import LinearRegression, RANSACRegressor
import numpy as np
from sklearn import cross_validation

# Initialize our algorithm class
alg = LinearRegression()

alg.fit(train[predictors],  train[targets])
predictions = alg.predict(train[predictors])

predictions[predictions<0]=0
predictions = np.around(predictions, 0)

# predictions = np.sum(predictions, axis=1)
predictions = np.concatenate(predictions, axis=0)
train['predict']=predictions

print(alg.coef_)
print("RMSLE:" RMSLE(train['predict'],train['count']))

10886
[[ -6.64673795e+12  -6.64673795e+12  -6.64673795e+12 ...,  -6.64673795e+12
   -6.64673795e+12  -6.64673795e+12]]
0.368907255692


## Create submission file

In [7]:
predictions = alg.predict(test[predictors])
test['predict']=predictions
submission = test[['datetime', 'predict']]
submission.columns=['datetime', 'count']
submission.to_csv("submission.csv", index=False)